In [1]:
from torch import nn

In [32]:
!pip install fastai

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 189 kB 3.9 MB/s eta 0:00:01
     |████████████████████████████████| 881.9 MB 26.4 MB/s eta 0:00:014    |██████████▋                     | 292.7 MB 5.4 MB/s eta 0:01:50     |█████████████████▋              | 484.4 MB 5.8 MB/s eta 0:01:09     |██████████████████▍             | 505.9 MB 19.4 MB/s eta 0:00:20
     |████████████████████████████████| 56 kB 24.4 MB/s eta 0:00:01
     |████████████████████████████████| 23.3 MB 31.3 MB/s eta 0:00:01
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0a0+b6df043
    Uninstalling torch-1.11.0a0+b6df043:
      Successfully uninstalled torch-1.11.0a0+b6df043
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.11.0a0
    Uninstalling torchvision-0.11.0a0:
      Successfully uninstalled torchvision-0.11.0a0
ERROR: pip's dependency resolver does not currently take into account all the pa

In [33]:
from fastai.vision.learner import create_head

/opt/conda/lib/python3.8/site-packages/torchvision/io/image.py:11: UserWarning: Failed to load image Python extension: /opt/conda/lib/python3.8/site-packages/torchvision/image.so: undefined symbol: _ZN3c105ErrorC2ENS_14SourceLocationESs
  warn(f"Failed to load image Python extension: {e}")


In [44]:
fastai_head = create_head(1536, 1)
fastai_head

Sequential(
  (0): AdaptiveConcatPool2d(
    (ap): AdaptiveAvgPool2d(output_size=1)
    (mp): AdaptiveMaxPool2d(output_size=1)
  )
  (1): Flatten(full=False)
  (2): BatchNorm1d(3072, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (3): Dropout(p=0.25, inplace=False)
  (4): Linear(in_features=3072, out_features=512, bias=False)
  (5): ReLU(inplace=True)
  (6): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (7): Dropout(p=0.5, inplace=False)
  (8): Linear(in_features=512, out_features=1, bias=False)
)

In [30]:
def create_head(nf, n_out, lin_ftrs=None, ps=0.5, concat_pool=True, first_bn=True, bn_final=False,
                lin_first=False, y_range=None):
    "Model head that takes `nf` features, runs through `lin_ftrs`, and out `n_out` classes."
    if concat_pool:
        nf *= 2
    lin_ftrs = [nf, 512, n_out] if lin_ftrs is None else [nf] + lin_ftrs + [n_out]
    bns = [first_bn] + [True]*len(lin_ftrs[1:])
    ps = L(ps)
    if len(ps) == 1:
        ps = [ps[0]/2] * (len(lin_ftrs)-2) + ps
    actns = [nn.ReLU(inplace=True)] * (len(lin_ftrs)-2) + [None]
    pool = AdaptiveConcatPool2d() if concat_pool else nn.AdaptiveAvgPool2d(1)
    layers = [pool, Flatten()]
    if lin_first:
        layers.append(nn.Dropout(ps.pop(0)))
    for ni,no,bn,p,actn in zip(lin_ftrs[:-1], lin_ftrs[1:], bns, ps, actns):
        layers += LinBnDrop(ni, no, bn=bn, p=p, act=actn, lin_first=lin_first)
    if lin_first:
        layers.append(nn.Linear(lin_ftrs[-2], n_out))
    if bn_final:
        layers.append(nn.BatchNorm1d(lin_ftrs[-1], momentum=0.01))
    if y_range is not None:
        layers.append(SigmoidRange(*y_range))
    return nn.Sequential(*layers)

In [31]:
create_head(1536, 1)

TypeError: 'float' object is not iterable

In [2]:
class AdaptiveConcatPool2d(nn.Module):
    def __init__(self, sz=None):
        super().__init__()
        sz = sz or (1,1)
        self.ap = nn.AdaptiveAvgPool2d(sz)
        self.mp = nn.AdaptiveMaxPool2d(sz)
    def forward(self, x): return torch.cat([self.mp(x), self.ap(x)], 1)
    

In [39]:
head = nn.Sequential(
    AdaptiveConcatPool2d(1),
    nn.Flatten(),
    nn.BatchNorm1d(3072),
    nn.Dropout(p=0.25),
    nn.Linear(in_features=3072, out_features=512),
    nn.ReLU(inplace=True),
    nn.BatchNorm1d(512),
    nn.Dropout(p=0.5),
    nn.Linear(in_features=512, out_features=10)
)
head

Sequential(
  (0): AdaptiveConcatPool2d(
    (ap): AdaptiveAvgPool2d(output_size=1)
    (mp): AdaptiveMaxPool2d(output_size=1)
  )
  (1): Flatten(start_dim=1, end_dim=-1)
  (2): BatchNorm1d(3072, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (3): Dropout(p=0.25, inplace=False)
  (4): Linear(in_features=3072, out_features=512, bias=True)
  (5): ReLU(inplace=True)
  (6): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (7): Dropout(p=0.5, inplace=False)
  (8): Linear(in_features=512, out_features=10, bias=True)
)

In [45]:
import torch 

x = torch.randn(8, 1, 1536)
fastai_head(x).shape

RuntimeError: running_mean should contain 2 elements not 3072